# Waddinton OT trajectory analysis


In [35]:
import importlib
importlib.reload(scrabbit.traj)

<module 'scrabbit.traj' from 'g:\\my drive\\postgrad\\phd\\projects\\rabbit-gastrulation-atlas\\rabbitgastrulation2021\\scrabbitpy\\src\\scrabbit\\traj.py'>

In [8]:
import scrabbit
from matplotlib import pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import Box,fixed,interact,interactive,HBox,Layout
import scanpy as sc
import pandas as pd
import wot
import scipy
import anndata

## Load trajectories

In [2]:
r_data = sc.read_h5ad("../data-in/rabbit/anndata.h5ad")
r_data

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


AnnData object with n_obs × n_vars = 146133 × 30725
    obs: 'cell', 'barcode', 'sample', 'stage', 'batch', 'doub.density', 'doublet', 'stripped', 'cluster', 'sizeFactor', 'celltype', 'singler', 'leiden_res1', 'leiden_res8', 'anatomical_loc', 'day'
    var: 'ensembl_ids'
    uns: 'anatomical_loc_colors', 'celltype_colors', 'leiden', 'neighbors', 'singler_colors', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    layers: 'logcounts'
    obsp: 'connectivities', 'distances'

In [11]:
r_traj = scrabbit.io.loadWOTCSV("../data-test/rabbit/trajectory_analysis/trajectory/")
r_fates = scrabbit.io.loadWOTCSV("../data-test/rabbit/trajectory_analysis/fates/")

In [12]:
r_traj.obsm["X_umap"] = r_data.obsm["X_umap"]
r_fates.obsm["X_umap"] = r_data.obsm["X_umap"]

### Visualise trajectories


In [13]:
rabbit_dropdown = widgets.Dropdown(
    options=r_traj.var.index,
    description='Trajectory:'
)

def plotWOTTrajectory(adata, celltype):
    sc.pl.umap(adata,color=celltype,color_map="viridis_r")

traj_widget = interactive(plotWOTTrajectory, adata=fixed(r_traj), celltype=rabbit_dropdown)
fate_widget = interactive(plotWOTTrajectory, adata=fixed(r_fates), celltype=rabbit_dropdown)

items = [traj_widget,fate_widget]
HBox(items,layout=Layout(flex_flow="flex-inline"))

## Extract trajectories

In [36]:
dropdown = widgets.Dropdown(
    options=r_traj.var.index,
    description='Trajectory:'
)

slider = widgets.FloatSlider(
    value=0.95,
    min=0.75,
    max=0.99,
    step=0.01
)

colour_dropdown = widgets.Dropdown(
    options=["celltype","stage"],
    description='Colour by:'
)

def plotExtractedTrajectory(adata, traj, celltype, thresh = 0.95, colour_by="celltype"):
    out = scrabbit.traj.extractTrajectory(adata, traj, celltype = celltype, thresh = thresh)
    sc.pl.umap(out,color=colour_by)

interactive(plotExtractedTrajectory, adata=fixed(r_data), traj=fixed(r_traj), celltype=dropdown, thresh=slider, colour_by = colour_dropdown)


interactive(children=(Dropdown(description='Trajectory:', options=('Allantois', 'Allantois endothelium', 'Amni…